In [1]:
%pip install jmespath
%pip install plotly


Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 59.6 MB/s eta 0:00:00m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import getpass
import yaml
import json
import pandas as pd
import jmespath
import plotly.express as px
import requests
import nest_asyncio
from nomad.client import ArchiveQuery
nest_asyncio.apply()
password = getpass.getpass()


ModuleNotFoundError: No module named 'yaml'

In [6]:
# username = "brueckner"
username = "aalbino"
response = requests.get(
    "https://nomad-lab.eu/prod/v1/staging/api/v1/auth/token?username={}&password={}".format(username, password))
# 'https://nomad-lab.eu/prod/api/v1/auth/token', params=dict(username='brueckner', password=password))
token = response.json()['access_token']


In [8]:
base_url = "https://nomad-lab.eu/prod/v1/staging/api/v1" # this is for the beta; otherwise default in ArchiveQuery is enough


In [ ]:

query = {
    "results.material.elements:all": [
        "Si"
    ],
    "results.material.n_elements": {
        "gte": 1,
        "lte": 1
    },
    "results.method.simulation.gw.type:any": [
        "G0W0"
    ],
    "results.method.simulation.gw.starting_point_type:any": [
        "LDA"
    ],
    "results.properties.available_properties:all": [
        "dos_electronic"
    ]
}
required = {
    'results': 'include-resolved',
    'run': 'include-resolved'
}

archives = ArchiveQuery(url=base_url, query=query, required=required).download()
print(archives)

In [9]:
def get_entries_of_upload(uploadid):
    selected_entries = []
    query = {
        'required': {
            'metadata': '*', }
    }
    response = requests.get(
        f'{base_url}/uploads/{uploadid}/entries',
        headers={'Authorization': f'Bearer {token}'}, json=query)
    response.json()["data"][0]["entry_id"]  # .keys()#['data']
    for e in response.json()["data"]:
        if e['entry_metadata']['entry_type'] != 'Schema':
            selected_entries.append(e["entry_id"])
            # print(e["entry_id"])
    return selected_entries


In [10]:
uploadid = "Qh1zOrsgSPK4_X4VMKhdCw"
selected_entries = get_entries_of_upload(uploadid)


In [12]:
# entry="CKxZBTPxGD2Oly2-vnDlk8Iwt_sM"
def get_entry_archive_data(entry):
    query = {
        'required': {
            'data': "*", }
    }

    response = requests.post(f'{base_url}/entries/{entry}/archive/query',
                             headers={'Authorization': f'Bearer {token}'}, json=query)
    return response.json()["data"]


# Plot and analysis 1: GAaO phase
1)  #009 - #010 - #011 - #012
change of growth temperature to study the stabilization of the GAaO phase - they can be helpful for plotting/analyzing the temperature ramps


In [ ]:
dfs = []
for entry in selected_entries[:4]:
    # print(entry)
    sample = get_entry_archive_data(entry)
    if 'measurements' in sample["archive"]["data"]['HRXRD'].keys():
        df = pd.concat([(pd.DataFrame.from_dict(sample["archive"]["data"]['overview'], orient='index',).T[['sample',  'film',
                                                                                                           'substrate', 'substrate_temperature', 'carrier_gas', 'VI_III_ratio',
                                                                                                           'growth_time', ]]),
                        (pd.DataFrame(sample["archive"]
                                      ["data"]['HRXRD']['measurements'])),
                        ], axis=1)  # .fillna(method='ffill', axis=0))
        print(df)
        df[['sample',
            'film', 'substrate',
            'substrate_temperature',
            'carrier_gas',
            'VI_III_ratio',
            'growth_time',]] = df[['sample',
                                   'film', 'substrate',
                                   'substrate_temperature',
                                   'carrier_gas',
                                   'VI_III_ratio',
                                   'growth_time',]].fillna(method='ffill', axis=0)
        dfs.append(df)
print(dfs)

In [14]:
pd.concat(dfs).columns


Index(['sample', 'film', 'substrate', 'substrate_temperature', 'carrier_gas',
       'VI_III_ratio', 'growth_time', 'name', 'datetime', 'lab_id',
       'sample_id', 'phase', 'peak_fwhm_rocking_curve'],
      dtype='object')

In [15]:
pd.concat(dfs).reset_index()[['sample', 'film', 'substrate', 'substrate_temperature', 'carrier_gas',
                              'VI_III_ratio', 'growth_time',
                              'sample_id', 'phase', 'peak_fwhm_rocking_curve']]


,sample,film,substrate,substrate_temperature,carrier_gas,VI_III_ratio,growth_time,sample_id,phase,peak_fwhm_rocking_curve
0,9,Ga2O3,sapphire (001),650.0,Argon,117.182771316363,30.0,9,kappa,0.55
1,9,Ga2O3,sapphire (001),650.0,Argon,117.182771316363,30.0,9,kappa,0.64
2,9,Ga2O3,sapphire (001),650.0,Argon,117.182771316363,30.0,9,kappa,0.56
3,10,Ga2O3,sapphire (001),700.0,Argon,117.182771316363,30.0,10,kappa,NaN
4,10,Ga2O3,sapphire (001),700.0,Argon,117.182771316363,30.0,10,kappa,NaN
5,11,Ga2O3,sapphire (001),750.0,Argon,117.182771316363,30.0,11,kappa,NaN
6,11,Ga2O3,sapphire (001),750.0,Argon,117.182771316363,30.0,11,beta,NaN
7,12,Ga2O3,sapphire (001),800.0,Argon,117.182771316363,30.0,12,beta,NaN
8,12,Ga2O3,sapphire (001),800.0,Argon,117.182771316363,30.0,12,NaN,NaN


# conclusion 
T<750 only kappa

T > 750°C on kappa and beta phase possible --> different temperature center  to edge?

T = 800 only beta

# Plot2: thicknesses homogeneity
2) #013 - #014 - #023 - #024
change of Manual Valves (Inner/Outer) (yellow block in GrowthRun Sheet) to study effect of thicknesses homogeneity in different points of the substrate (data in AFMReflectanceSheet)  - some plots of thick vs. Valves could be extracted from these.


In [18]:
dfs = []
for entry in selected_entries[4:]:
    # print(entry)
    sample = get_entry_archive_data(entry)  # ["archive"]["data"]['overview']
    if 'measurements' in sample["archive"]["data"]['reflectance'].keys() and 'growth_run_steps' in sample["archive"]["data"]['growth_run'].keys():
        # print((pd.DataFrame(pd.DataFrame(sample["archive"]["data"]['reflectance']['measurements']))))
        dfs.append(pd.concat([(pd.DataFrame(pd.DataFrame(sample["archive"]["data"]['reflectance']['measurements']))),
                              pd.DataFrame(sample["archive"]["data"]['growth_run']['growth_run_steps'])[['temperature', 'metal_inner_valve', 'metal_outer_valve',]][:1]],
                             axis=1).fillna(method='ffill', axis=0))


In [19]:
pd.concat(dfs).columns  # .drop(["datetime"])


Index(['datetime', 'description', 'sample_id', 'thickness', 'growth_rate',
       'temperature', 'metal_inner_valve', 'metal_outer_valve'],
      dtype='object')

In [102]:
df_homogenuity = pd.concat(dfs)[['sample_id', 'description', 'thickness',
                                'growth_rate', 'temperature', 'metal_inner_valve', 'metal_outer_valve']].reset_index(drop=True)


In [103]:
# df_homogenuity=df_homogenuity.loc[df_homogenuity.sample_id.isin(["23","24"])] # filter only some of the samples


In [104]:
df_homogenuity["metal inner - outer valve flow"] = df_homogenuity["metal_inner_valve"].astype(
    int).astype(str) + " - "+df_homogenuity["metal_outer_valve"].astype(int).astype(str)


In [112]:
df_homogenuity.description.unique()


array(['center', 'mid-center', 'mid-out', 'out', 'center 1', 'center 2',
       'mid 1', 'mid 2'], dtype=object)

In [113]:
# df_homogenuity = df_homogenuity.sort_values(by='description', key=lambda x: x.map({v: i for i, v in enumerate(['center', 'center 1', 'center 2', 'mid-center', 'mid 1', 'mid 2', 'mid-out', 'out',
#      ])}))


In [114]:
df_homogenuity


,sample_id,description,thickness,growth_rate,temperature,metal_inner_valve,metal_outer_valve,metal inner - outer valve flow
0,14,center,910.0,30.333333,650.0,50.0,150.0,50 - 150
1,14,mid-center,880.0,29.333333,650.0,50.0,150.0,50 - 150
2,14,mid-out,760.0,25.333333,650.0,50.0,150.0,50 - 150
3,14,out,740.0,24.666667,650.0,50.0,150.0,50 - 150
4,23,center 1,750.0,25.000000,650.0,45.0,155.0,45 - 155
5,23,center 2,710.0,23.666667,650.0,45.0,155.0,45 - 155
6,23,mid 1,950.0,31.666667,650.0,45.0,155.0,45 - 155
7,23,mid 2,850.0,28.333333,650.0,45.0,155.0,45 - 155
8,23,out,770.0,25.666667,650.0,45.0,155.0,45 - 155
9,24,center 1,960.0,32.000000,650.0,48.0,152.0,48 - 152


In [115]:
import plotly.io as pio
pio.renderers.default = 'iframe'


In [127]:
# fig = px.line(data_frame=df_homogenuity.loc[df_homogenuity.sample_id.isin(["23","24"])], x="description", y="growth_rate", color="metal inner - outer valve flow")
fig = px.line(data_frame=df_homogenuity,
              x="description",
              y="growth_rate",
              color="sample_id",
              category_orders={"description": [
                  'center', 'center 1', 'center 2', 'mid-center', 'mid 1', 'mid 2', 'mid-out', 'out']},
              markers=True
              )
fig.update_traces(marker=dict(size=12))
fig.show()


## Problem: description not uniform!! Input from Luca neededreset_index

# testing etc

In [33]:
pd.concat([(pd.DataFrame(pd.DataFrame(sample["archive"]["data"]['reflectance']['measurements']))),
           pd.DataFrame(sample["archive"]["data"]['growth_run']['growth_run_steps'])[['temperature', 'metal_inner_valve', 'metal_outer_valve',]][:1]],
          axis=1).fillna(method='ffill', axis=0)


,datetime,description,sample_id,thickness,growth_rate,temperature,metal_inner_valve,metal_outer_valve
0,2023-05-02T00:00:00+00:00,center 1,24,960.0,32.000000,650.0,48.0,152.0
1,2023-05-02T00:00:00+00:00,center 2,24,780.0,26.000000,650.0,48.0,152.0
2,2023-05-02T00:00:00+00:00,mid 1,24,1010.0,33.666667,650.0,48.0,152.0
3,2023-05-02T00:00:00+00:00,mid 2,24,950.0,31.666667,650.0,48.0,152.0
4,2023-05-02T00:00:00+00:00,out,24,800.0,26.666667,650.0,48.0,152.0


In [34]:
pd.concat(dfs)


,datetime,description,sample_id,thickness,growth_rate,temperature,metal_inner_valve,metal_outer_valve
0,2023-03-28T00:00:00+00:00,center,14,910.0,30.333333,650.0,50.0,150.0
1,2023-03-28T00:00:00+00:00,mid-center,14,880.0,29.333333,650.0,50.0,150.0
2,2023-03-28T00:00:00+00:00,mid-out,14,760.0,25.333333,650.0,50.0,150.0
3,2023-03-28T00:00:00+00:00,out,14,740.0,24.666667,650.0,50.0,150.0
0,NaN,center 1,23,750.0,25.000000,650.0,45.0,155.0
1,NaN,center 2,23,710.0,23.666667,650.0,45.0,155.0
2,NaN,mid 1,23,950.0,31.666667,650.0,45.0,155.0
3,NaN,mid 2,23,850.0,28.333333,650.0,45.0,155.0
4,NaN,out,23,770.0,25.666667,650.0,45.0,155.0
0,2023-05-02T00:00:00+00:00,center 1,24,960.0,32.000000,650.0,48.0,152.0


In [35]:
pd.DataFrame(sample["archive"]["data"]['growth_run']['growth_run_steps'])[
    ['temperature', 'metal_inner_valve', 'metal_outer_valve',]][:1]  # .index#columns#[[]]#.keys()#['reflectance']


,temperature,metal_inner_valve,metal_outer_valve
0,650.0,48,152


In [36]:
sample["archive"]["data"]['reflectance']


{'fill_archive_from_datafile': False,
 'data_file': '024.xlsx',
 'measurements': [{'datetime': '2023-05-02T00:00:00+00:00',
   'description': 'center 1',
   'sample_id': '24',
   'thickness': 960.0,
   'growth_rate': 32.0},
  {'datetime': '2023-05-02T00:00:00+00:00',
   'description': 'center 2',
   'sample_id': '24',
   'thickness': 780.0,
   'growth_rate': 26.0},
  {'datetime': '2023-05-02T00:00:00+00:00',
   'description': 'mid 1',
   'sample_id': '24',
   'thickness': 1010.0,
   'growth_rate': 33.6666666666667},
  {'datetime': '2023-05-02T00:00:00+00:00',
   'description': 'mid 2',
   'sample_id': '24',
   'thickness': 950.0,
   'growth_rate': 31.6666666666667},
  {'datetime': '2023-05-02T00:00:00+00:00',
   'description': 'out',
   'sample_id': '24',
   'thickness': 800.0,
   'growth_rate': 26.6666666666667}]}

In [45]:
print(sample["archive"]["data"]["HRXRD"])


{'fill_archive_from_datafile': False, 'data_file': '024.xlsx'}


In [41]:
if 'measurements' in sample["archive"]["data"]['reflectance'].keys():
    pd.concat([(pd.DataFrame(sample["archive"]["data"]['HRXRD']['measurements']).dropna(axis=0)), (pd.DataFrame.from_dict(sample["archive"]["data"]['overview'], orient='index',).T[['sample',  'film',
                                                                                                                                                                                     'substrate', 'substrate_temperature', 'carrier_gas', 'VI_III_ratio',
                                                                                                                                                                                     'growth_time', ]])], axis=1).fillna(method='ffill', axis=0)


KeyError: 'measurements'

In [39]:
# ["archive"]["data"]['overview']
sample24 = get_entry_archive_data("8dc9SBveR53fHCDI7zkAPu1uUT7l")


In [12]:
sample24["archive"]["data"]['overview']


{'fill_archive_from_datafile': False,
 'sample': '10',
 'date': '2023-03-21T00:00:00+00:00',
 'operator': 'IMEM',
 'film': 'Ga2O3',
 'substrate': 'sapphire (001)',
 'substrate_temperature': 700.0,
 'carrier_gas': 'Argon',
 'VI_III_ratio': '117.182771316363',
 'growth_time': 30.0,
 'notes': 'test with increased temperature to appreach beta phase',
 'data_file': '010.xlsx'}

In [13]:
sample24["archive"]["data"].keys()  # ['overview']


dict_keys(['m_def', 'fill_archive_from_datafile', 'name', 'datetime', 'lab_id', 'data_file', 'overview', 'precursors', 'substrate', 'MIST', 'pregrowth', 'growth_run', 'samples_cut', 'contacts_sputtering', 'SEM', 'AFM', 'reflectance', 'UVabsorbance', 'hall', 'HRXRD'])

In [14]:
sample24["archive"]["data"]['SEM']


{'fill_archive_from_datafile': False,
 'data_file': '010.xlsx',
 'measurements': [{'name': '010',
   'datetime': '2023-03-21T00:00:00+00:00',
   'lab_id': 'CR21UP0001-0025',
   'sample_id': '10.0'},
  {'name': '010',
   'datetime': '2023-03-21T00:00:00+00:00',
   'lab_id': 'CR21UP0001-0025',
   'sample_id': '10.0'},
  {'name': '010',
   'datetime': '2023-03-21T00:00:00+00:00',
   'lab_id': 'CR21UP0001-0025',
   'sample_id': '10.0'},
  {'name': '010',
   'datetime': '2023-05-26T10:20:56.376190+00:00',
   'lab_id': 'CR21UP0001-0025'}]}

In [15]:
sample24["archive"]["data"]['HRXRD']


{'fill_archive_from_datafile': False,
 'data_file': '010.xlsx',
 'measurements': [{'name': '010',
   'datetime': '2023-05-26T10:20:56.386945+00:00',
   'lab_id': 'CR21UP0001-0025',
   'sample_id': '10',
   'phase': 'kappa'},
  {'name': '010',
   'datetime': '2023-05-26T10:20:56.388238+00:00',
   'lab_id': 'CR21UP0001-0025',
   'sample_id': '10',
   'phase': 'kappa'}]}

In [16]:
sample24["archive"]["data"]['UVabsorbance']


{'fill_archive_from_datafile': False, 'data_file': '010.xlsx'}

In [18]:
(pd.DataFrame(sample["archive"]["data"]
 ['reflectance']['measurements']).dropna(axis=0))


,datetime,description,sample_id,growth_rate
0,2023-03-23T00:00:00+00:00,center,12,0.0
1,2023-03-23T00:00:00+00:00,mid,12,0.0
2,2023-03-23T00:00:00+00:00,out,12,0.0


In [46]:
pd.DataFrame.from_dict(sample["archive"]["data"]['overview'], orient='index',).T[['sample',  'film',
                                                                                  'substrate', 'substrate_temperature', 'carrier_gas', 'VI_III_ratio',
                                                                                  'growth_time', ]]


,sample,film,substrate,substrate_temperature,carrier_gas,VI_III_ratio,growth_time
0,9,Ga2O3,sapphire (001),650.0,Argon,117.182771316363,30.0


In [52]:
sample = get_entry_archive_data('VhE6Q0VtyppepthvMItPmGF_rj47')
pd.concat([(pd.DataFrame(sample["archive"]["data"]['HRXRD']['measurements']).dropna(axis=0)), (pd.DataFrame.from_dict(sample["archive"]["data"]['overview'], orient='index',).T[['sample',  'film',
                                                                                                                                                                                 'substrate', 'substrate_temperature', 'carrier_gas', 'VI_III_ratio',
                                                                                                                                                                                 'growth_time', ]])], axis=1).fillna(method='ffill', axis=0)


,name,datetime,lab_id,sample_id,phase,peak_fwhm_rocking_curve,sample,film,substrate,substrate_temperature,carrier_gas,VI_III_ratio,growth_time
0,009,2023-06-01T19:22:08.473593+00:00,CR21UP0001-0025,9,kappa,0.55,9,Ga2O3,sapphire (001),650.0,Argon,117.182771316363,30.0
1,009,2023-06-01T19:22:08.474942+00:00,CR21UP0001-0025,9,kappa,0.64,9,Ga2O3,sapphire (001),650.0,Argon,117.182771316363,30.0
2,009,2023-06-01T19:22:08.476087+00:00,CR21UP0001-0025,9,kappa,0.56,9,Ga2O3,sapphire (001),650.0,Argon,117.182771316363,30.0


In [28]:
for entry in selected_entries:
    print(entry)
    sample = get_entry_archive_data(entry)  # ["archive"]["data"]['overview']
    if 'measurements' in sample["archive"]["data"]['HRXRD'].keys():
        print(pd.DataFrame(sample["archive"]["data"]
              ['HRXRD']['measurements']).dropna(axis=0))


VhE6Q0VtyppepthvMItPmGF_rj47
  name                          datetime           lab_id sample_id  phase  \
0  009  2023-05-26T10:20:56.521356+00:00  CR21UP0001-0025         9  kappa   
1  009  2023-05-26T10:20:56.522452+00:00  CR21UP0001-0025         9  kappa   
2  009  2023-05-26T10:20:56.523435+00:00  CR21UP0001-0025         9  kappa   

  peak_fwhm_rocking_curve  
0                    0.55  
1                    0.64  
2                    0.56  
8dc9SBveR53fHCDI7zkAPu1uUT7l
  name                          datetime           lab_id sample_id  phase
0  010  2023-05-26T10:20:56.386945+00:00  CR21UP0001-0025        10  kappa
1  010  2023-05-26T10:20:56.388238+00:00  CR21UP0001-0025        10  kappa
CKxZBTPxGD2Oly2-vnDlk8Iwt_sM
  name                          datetime           lab_id sample_id  phase
0  011  2023-05-26T10:38:48.585211+00:00  CR21UP0001-0025        11  kappa
1  011  2023-05-26T10:38:48.586529+00:00  CR21UP0001-0025        11   beta
WEsrUplQ8Cgc5NGFY1Bv-FN1IQkF
  sample_

In [21]:
import plotly.io as pio
pio.renderers.default = 'iframe'


In [22]:
fig = px.scatter(data_frame=(pd.DataFrame(sample["archive"]["data"]['reflectance']['measurements']).dropna(
    axis=0)), x="description", y="growth_rate")
fig.show()


In [23]:
for entry in selected_entries[:4]:
    print(entry)
    sample = get_entry_archive_data(entry)  # ["archive"]["data"]['overview']
    print(pd.DataFrame(sample["archive"]["data"]
          ['reflectance']['measurements']).dropna(axis=0))
    fig = px.scatter(data_frame=(pd.DataFrame(sample["archive"]["data"]['reflectance']['measurements']).dropna(
        axis=0)), x="description", y="growth_rate")
    fig.show()


VhE6Q0VtyppepthvMItPmGF_rj47
  name                   datetime           lab_id description sample_id  \
0  009  2023-03-15T00:00:00+00:00  CR21UP0001-0025      center         9   
1  009  2023-03-15T00:00:00+00:00  CR21UP0001-0025         mid         9   
2  009  2023-03-15T00:00:00+00:00  CR21UP0001-0025         out         9   

   thickness  growth_rate  
0     1430.0    47.666667  
1     1150.0    38.333333  
2      810.0    27.000000  


8dc9SBveR53fHCDI7zkAPu1uUT7l
  name                   datetime           lab_id description sample_id  \
0  010  2023-03-21T00:00:00+00:00  CR21UP0001-0025      center      10.0   
1  010  2023-03-21T00:00:00+00:00  CR21UP0001-0025         mid      10.0   
2  010  2023-03-21T00:00:00+00:00  CR21UP0001-0025         out      10.0   

   thickness  growth_rate  
0     1530.0    51.000000  
1     1010.0    33.666667  
2      740.0    24.666667  


CKxZBTPxGD2Oly2-vnDlk8Iwt_sM
  name                   datetime           lab_id description sample_id  \
0  011  2023-03-22T00:00:00+00:00  CR21UP0001-0025      center      11.0   
1  011  2023-03-22T00:00:00+00:00  CR21UP0001-0025         mid      11.0   
2  011  2023-03-22T00:00:00+00:00  CR21UP0001-0025         out      11.0   

   thickness  growth_rate  
0     1500.0    50.000000  
1      950.0    31.666667  
2      200.0     6.666667  


WEsrUplQ8Cgc5NGFY1Bv-FN1IQkF
                    datetime description sample_id  growth_rate
0  2023-03-23T00:00:00+00:00      center        12          0.0
1  2023-03-23T00:00:00+00:00         mid        12          0.0
2  2023-03-23T00:00:00+00:00         out        12          0.0


In [207]:
sample24["archive"]["data"]['overview']


{'fill_archive_from_datafile': False,
 'sample': '10',
 'date': '2023-03-21T00:00:00+00:00',
 'operator': 'IMEM',
 'film': 'Ga2O3',
 'substrate': 'sapphire (001)',
 'substrate_temperature': 700.0,
 'carrier_gas': 'Argon',
 'VI_III_ratio': '117.182771316363',
 'growth_time': 30.0,
 'notes': 'test with increased temperature to appreach beta phase',
 'data_file': '010.xlsx'}

In [201]:
sample24["archive"]["data"]['AFM']


{'fill_archive_from_datafile': False,
 'data_file': '010.xlsx',
 'measurements': [{'name': '010',
   'datetime': '2023-03-21T00:00:00+00:00',
   'lab_id': 'CR21UP0001-0025',
   'sample_id': '10.0'},
  {'name': '010',
   'datetime': '2023-03-21T00:00:00+00:00',
   'lab_id': 'CR21UP0001-0025',
   'sample_id': '10.0'},
  {'name': '010',
   'datetime': '2023-03-21T00:00:00+00:00',
   'lab_id': 'CR21UP0001-0025',
   'sample_id': '10.0'},
  {'name': '010', 'lab_id': 'CR21UP0001-0025'}]}

In [58]:
entrydata = response.json()["data"]  # ["archive"]["data"]#.keys()


In [88]:
entrydata["archive"]["data"].keys()


dict_keys(['m_def', 'fill_archive_from_datafile', 'name', 'datetime', 'lab_id', 'data_file', 'overview', 'precursors', 'substrate', 'MIST', 'pregrowth', 'growth_run', 'samples_cut', 'contacts_sputtering', 'SEM', 'AFM', 'reflectance', 'UVabsorbance', 'hall', 'HRXRD'])

In [97]:
entrydata["archive"]["data"]['overview']  # ['measurements']


{'fill_archive_from_datafile': False,
 'sample': '11',
 'date': '2023-03-22T00:00:00+00:00',
 'operator': 'IMEM',
 'film': 'Ga2O3',
 'substrate': 'sapphire (001)',
 'substrate_temperature': 750.0,
 'carrier_gas': 'Argon',
 'VI_III_ratio': '117.182771316363',
 'growth_time': 30.0,
 'notes': 'test with increased temperature to approach beta phase',
 'data_file': '011.xlsx'}

In [89]:
df_xrd = pd.DataFrame(entrydata["archive"]["data"]
                      ['HRXRD']['measurements']).dropna(axis=0)  # .keys())
df_xrd


,name,datetime,lab_id,sample_id,phase
0,011,2023-05-26T10:38:48.585211+00:00,CR21UP0001-0025,11,kappa
1,011,2023-05-26T10:38:48.586529+00:00,CR21UP0001-0025,11,beta


In [85]:
df_afm = pd.DataFrame(entrydata["archive"]["data"]
                      ['AFM']['measurements']).dropna(axis=0)  # .keys())
df_afm


,name,datetime,lab_id,sample_id
0,011,2023-03-22T00:00:00+00:00,CR21UP0001-0025,11.0
1,011,2023-03-22T00:00:00+00:00,CR21UP0001-0025,11.0
2,011,2023-03-22T00:00:00+00:00,CR21UP0001-0025,11.0


In [76]:
df_reflectance = pd.DataFrame(
    entrydata["archive"]["data"]["reflectance"]['measurements'],).dropna(axis=0)  # .keys()
df_reflectance


,name,datetime,lab_id,description,sample_id,thickness,growth_rate
0,011,2023-03-22T00:00:00+00:00,CR21UP0001-0025,center,11.0,1500.0,50.000000
1,011,2023-03-22T00:00:00+00:00,CR21UP0001-0025,mid,11.0,950.0,31.666667
2,011,2023-03-22T00:00:00+00:00,CR21UP0001-0025,out,11.0,200.0,6.666667


In [8]:
jmespath.search('data.name', entrydata)


'010'

In [12]:
jmespath.search("data.overview", MOVPE_data)


{'fill_archive_from_datafile': False,
 'sample': '10',
 'date': '2023-03-21T00:00:00+00:00',
 'operator': 'IMEM',
 'film': 'Ga2O3',
 'substrate': 'sapphire (001)',
 'substrate_temperature': 700,
 'carrier_gas': 'Argon',
 'VI_III_ratio': '117.18277131636286',
 'growth_time': 30,
 'notes': 'test with increased temperature to appreach beta phase',
 'data_file': '010.xlsx'}

In [3]:
path


'baz'

In [5]:
with open("010.archive.json") as stream:
    MOVPE_data = yaml.safe_load(stream)


In [6]:
MOVPE_data


{'data': {'m_def': '../upload/raw/movpe_cnr.schema.archive.yaml#/definitions/section_definitions/0',
  'fill_archive_from_datafile': False,
  'name': '010',
  'datetime': '2023-05-24T12:49:12.623621+00:00',
  'data_file': '010.xlsx',
  'overview': {'fill_archive_from_datafile': False,
   'sample': '10',
   'date': '2023-03-21T00:00:00+00:00',
   'operator': 'IMEM',
   'film': 'Ga2O3',
   'substrate': 'sapphire (001)',
   'substrate_temperature': 700,
   'carrier_gas': 'Argon',
   'VI_III_ratio': '117.18277131636286',
   'growth_time': 30,
   'notes': 'test with increased temperature to appreach beta phase',
   'data_file': '010.xlsx'},
  'precursors': {'data_file': '010.xlsx'},
  'substrate': {'fill_archive_from_datafile': False,
   'data_file': '010.xlsx',
   'substrates': [{'name': 'sapphire',
     'lab_id': 'CR21UP0001-0025',
     'alias': 'S010',
     'supplier': 'Cryscore',
     'orientation': '1',
     'off_cut': 0,
     'size': "2''",
     'susceptor': 'Metal [Nickel Alloy]',
  

In [27]:
pd.DataFrame.from_dict(MOVPE_data["data"]["growth_run"]["GrowthRunSteps"])


,name,elapsed_time,step_number,temperature,pressure,flow_metal_carrier,metal_inner_valve,metal_outer_valve,flow_oxydant_carrier,oxydant_inner_valve,oxydant_outer_valve,purge_flow,uniform_valve,purge_valve,notes,Bubblers,GasSource,MistSource1
0,deposition,30,1,700,100,700,60,140,700,60,140,2000,100,100,deposition in reactor,"[{'material': 'TMG ', 'dilution': 3, 'inject':...","[{'material': 'O2', 'dilution_in_cylinder': 1,...","[{'material': '0', 'valve': False}]"
1,purge,2,2,700,100,700,60,140,700,60,140,2000,100,100,purge after deposition,"[{'material': 'TMG ', 'dilution': 3, 'inject':...","[{'material': 'O2', 'dilution_in_cylinder': 1,...","[{'material': '0', 'valve': False}]"
2,cooling 1,10,3,500,100,700,60,140,700,60,140,2000,100,100,cooling ramp 0.3 °C/sec,"[{'material': 'TMG ', 'dilution': 3, 'inject':...","[{'material': 'O2', 'dilution_in_cylinder': 1,...","[{'material': '0', 'valve': False}]"
3,cooling 2,50,4,70,100,500,60,140,500,60,140,1000,100,100,"natural cooling down, precursors flows in standby","[{'material': 'TMG ', 'dilution': 3, 'inject':...","[{'material': 'O2', 'dilution_in_cylinder': 1,...","[{'material': '0', 'valve': False}]"
4,open,0,5,50,1000,100,60,140,100,60,140,1000,100,100,open in reactor at 50°C,"[{'material': 'TMG ', 'dilution': 3, 'inject':...","[{'material': 'O2', 'dilution_in_cylinder': 1,...","[{'material': '0', 'valve': False}]"


change of Manual Valves (Inner/Outer) (yellow block in GrowthRun Sheet) to study effect of thicknesses homogeneity in different points of the substrate (data in AFMReflectanceSheet)  - some plots of thick vs. Valves could be extracted from these.

In [33]:
pd.DataFrame(MOVPE_data["data"]["AFM"]["measurements"])


,datetime,sample_id,thickness,growth_rate,notes
0,2023-03-21T00:00:00+00:00,10.0,1530.0,51.000000,center
1,2023-03-21T00:00:00+00:00,10.0,1010.0,33.666667,mid
2,2023-03-21T00:00:00+00:00,10.0,740.0,24.666667,out
3,NaN,NaN,NaN,0.000000,NaN


In [37]:
MOVPE_data["data"]["overview"]  # .keys()


{'fill_archive_from_datafile': False,
 'sample': '10',
 'date': '2023-03-21T00:00:00+00:00',
 'operator': 'IMEM',
 'film': 'Ga2O3',
 'substrate': 'sapphire (001)',
 'substrate_temperature': 700,
 'carrier_gas': 'Argon',
 'VI_III_ratio': '117.18277131636286',
 'growth_time': 30,
 'notes': 'test with increased temperature to appreach beta phase',
 'data_file': '010.xlsx'}

In [34]:
pd.DataFrame.from_dict(MOVPE_data["data"])  # .keys())


,m_def,fill_archive_from_datafile,name,datetime,data_file,overview,precursors,substrate,MIST,pregrowth,growth_run,samples_cut,AFM,HRXRD,contacts_sputtering,description
fill_archive_from_datafile,../upload/raw/movpe_cnr.schema.archive.yaml#/d...,False,010,2023-05-24T12:49:12.623621+00:00,010.xlsx,False,NaN,False,False,False,False,False,False,False,False,a
sample,../upload/raw/movpe_cnr.schema.archive.yaml#/d...,False,010,2023-05-24T12:49:12.623621+00:00,010.xlsx,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a
date,../upload/raw/movpe_cnr.schema.archive.yaml#/d...,False,010,2023-05-24T12:49:12.623621+00:00,010.xlsx,2023-03-21T00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a
operator,../upload/raw/movpe_cnr.schema.archive.yaml#/d...,False,010,2023-05-24T12:49:12.623621+00:00,010.xlsx,IMEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a
film,../upload/raw/movpe_cnr.schema.archive.yaml#/d...,False,010,2023-05-24T12:49:12.623621+00:00,010.xlsx,Ga2O3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a
substrate,../upload/raw/movpe_cnr.schema.archive.yaml#/d...,False,010,2023-05-24T12:49:12.623621+00:00,010.xlsx,sapphire (001),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a
substrate_temperature,../upload/raw/movpe_cnr.schema.archive.yaml#/d...,False,010,2023-05-24T12:49:12.623621+00:00,010.xlsx,700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a
carrier_gas,../upload/raw/movpe_cnr.schema.archive.yaml#/d...,False,010,2023-05-24T12:49:12.623621+00:00,010.xlsx,Argon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a
VI_III_ratio,../upload/raw/movpe_cnr.schema.archive.yaml#/d...,False,010,2023-05-24T12:49:12.623621+00:00,010.xlsx,117.18277131636286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a
growth_time,../upload/raw/movpe_cnr.schema.archive.yaml#/d...,False,010,2023-05-24T12:49:12.623621+00:00,010.xlsx,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a


In [21]:
MOVPE_data["data"]["growth_run"]["GrowthRunSteps"][0]


{'name': 'deposition',
 'elapsed_time': 30,
 'step_number': 1,
 'temperature': 800,
 'pressure': 100,
 'flow_metal_carrier': 700,
 'metal_inner_valve': 60,
 'metal_outer_valve': 140,
 'flow_oxydant_carrier': 700,
 'oxydant_inner_valve': 60,
 'oxydant_outer_valve': 140,
 'purge_flow': 2000,
 'uniform_valve': 100,
 'purge_valve': 100,
 'notes': 'deposition in reactor',
 'Bubblers': [{'material': 'TMG ',
   'dilution': 3,
   'inject': 30,
   'pressure': 1000,
   'temperature': 5,
   'partial_pressure': 0.0013825471361490558,
   'valve': True},
  {'material': 'H2O',
   'dilution': 0,
   'inject': 0,
   'pressure': 0,
   'temperature': 30,
   'partial_pressure': 0.1620107048894472,
   'valve': True}],
 'GasSource': [{'material': 'O2',
   'dilution_in_cylinder': 1,
   'flow_from_mfc': 0,
   'partial_pressure': 0,
   'cylinder_pressure': 160,
   'valve': False},
  {'material': 'SiH4',
   'dilution_in_cylinder': 0.0005,
   'flow_from_mfc': 0,
   'partial_pressure': 0,
   'cylinder_pressure': 0

In [13]:
MOVPE_data["data"]["growth_run"]["GrowthRunSteps"]  # [0][""]#.keys()


{'name': 'deposition',
 'elapsed_time': 30,
 'step_number': 1,
 'temperature': 800,
 'pressure': 100,
 'flow_metal_carrier': 700,
 'metal_inner_valve': 60,
 'metal_outer_valve': 140,
 'flow_oxydant_carrier': 700,
 'oxydant_inner_valve': 60,
 'oxydant_outer_valve': 140,
 'purge_flow': 2000,
 'uniform_valve': 100,
 'purge_valve': 100,
 'notes': 'deposition in reactor',
 'Bubblers': [{'material': 'TMG ',
   'dilution': 3,
   'inject': 30,
   'pressure': 1000,
   'temperature': 5,
   'partial_pressure': 0.0013825471361490558,
   'valve': True},
  {'material': 'H2O',
   'dilution': 0,
   'inject': 0,
   'pressure': 0,
   'temperature': 30,
   'partial_pressure': 0.1620107048894472,
   'valve': True}],
 'GasSource': [{'material': 'O2',
   'dilution_in_cylinder': 1,
   'flow_from_mfc': 0,
   'partial_pressure': 0,
   'cylinder_pressure': 160,
   'valve': False},
  {'material': 'SiH4',
   'dilution_in_cylinder': 0.0005,
   'flow_from_mfc': 0,
   'partial_pressure': 0,
   'cylinder_pressure': 0

In [5]:
MOVPE_data


{'data': {'m_def': '../upload/raw/movpe_cnr.schema.archive.yaml#/definitions/section_definitions/0',
  'fill_archive_from_datafile': False,
  'name': '012',
  'datetime': '2023-05-25T07:40:59.700075+00:00',
  'lab_id': 'CR21UP0001-0025',
  'description': 'aa',
  'data_file': '012.xlsx',
  'overview': {'fill_archive_from_datafile': False,
   'sample': '12',
   'date': '2023-03-23T00:00:00+00:00',
   'category': 'epitaxial growth synthesis',
   'operator': 'IMEM',
   'film': 'Ga2O3',
   'substrate': 'sapphire (001)',
   'substrate_temperature': 800,
   'carrier_gas': 'Argon',
   'VI_III_ratio': '117.18277131636286',
   'growth_time': 30,
   'notes': 'test with increased temperature to approach beta phase.',
   'data_file': '012.xlsx'},
  'precursors': {'data_file': '012.xlsx'},
  'substrate': {'fill_archive_from_datafile': False,
   'data_file': '012.xlsx',
   'substrates': [{'name': 'sapphire',
     'datetime': '2023-05-25T07:42:14.527564+00:00',
     'lab_id': 'CR21UP0001-0025',
     '